In [68]:
import pandas as pd

df = pd.read_excel("./feature/class.xlsx")
# print(df)
dataset_list = ["tongji", "xiangyang", "kits", "henan"]
for dataset in dataset_list:
    df_csv = pd.read_excel("./feature/ring/" + dataset + "_ring5.xlsx", index_col=0)
    namelist = df[(df["dataset"] == dataset) & (df["exclusion"] == 1)]["name"].values
    # 批量去掉.npy
    namelist = [name[:-4] for name in namelist]
    new_namelist = df_csv.index
    for name in namelist:
        if name in new_namelist:
            continue
        else:
            print(dataset, name)

    print(len(namelist))
    # tongji_list = namelist
    csv = df_csv.loc[namelist, :]
    print(csv.shape)
    csv.to_excel("./feature/ring/ring_" + dataset + ".xlsx")

706
(706, 144)
82
(82, 129)
167
(167, 129)
187
(187, 129)


In [67]:
tj1 = pd.read_csv("./feature/ring_5.csv", index_col=0)

tj2 = pd.read_excel("./feature/tongji_ring5.xlsx", index_col=0)
# tj2的index去掉.nii.gz
tj2.index = [name[:-7] for name in tj2.index]
print(tj1.index, tj2.index)
if "XU REN DOU" in tj2.index:
    if "XU REN DOU" in tj1.index:
        print("XU REN DOU")

# 查看tj1和tj2相同的index
same_index = tj1.index.intersection(tj2.index)
# 去掉tj2中相同的index
tj2 = tj2.drop(same_index)

# index去掉XU REN DOU项
# tj2 = tj2.drop("XU REN DOU", axis=0)
# print(tj1.index)
tj = pd.concat([tj1, tj2])
print(tj.shape)
tj.to_excel("./feature/rad/ring_tongji.xlsx", index=True)

Index(['RONG ZONG AN', 'TJH0334', 'TJH0346', 'TJH0811', 'GUO RAN FU',
       'YU TAO XIANG', 'JIN HAI YI', 'TJH0072', 'TJH0832', 'TJH0731',
       ...
       'ZHOU ZE MIN', 'XIE BIN', 'TJH0606', 'TJH0026', 'DENG XUAN YONG',
       'CHEN YANG GUI', 'WANG SHENG HUA', 'TJH0369', 'TJH0357', 'TJH0543'],
      dtype='object', length=706) Index(['FAN XUE YING', 'LIU WEI', 'LIU YING E', 'CHEN ZHI GANG', 'LIN RUI LAN',
       'WANG GUO BIN', 'WU HUI XIANG', 'DAN BI ZHONG', 'GAO HUI PING',
       'WANG YING ZHEN',
       ...
       'SHEN RUI', 'WANG ZU WEI', 'ZHENG MEI JIAO', 'LI YAN XIONG', 'YAO WEI',
       'BAO CHAO', 'QIU XIAO DONG', 'YI WEN HUA', 'XUE HONG', 'YAO SHANG YI'],
      dtype='object', length=578)
XU REN DOU
(941, 144)


In [45]:
tj = pd.read_excel("./feature/rad/rad_tongji.xlsx", index_col=0)
print(tj.shape)
tj_list = tj.index
print(len(tj_list))
print(len(tongji_list))
print(len(set(tj_list)))
missing_in_tongji = set(tj_list) - set(tongji_list)
print(missing_in_tongji)
# for name in tj_list:
#     if name in tongji_list:
#         continue
#     else:
#         print("tj_list", name)

(707, 144)
707
706
706
set()


In [46]:
# 方法1：使用collections.Counter
from collections import Counter

# 查找重复项及其出现次数
duplicates = [item for item, count in Counter(tj_list).items() if count > 1]
print("重复的项：")
for item in duplicates:
    print(f"{item}: 出现 {Counter(tj_list)[item]} 次")

# # 方法2：使用set比较
# print("重复的个数：", len(tongji_list) - len(set(tongji_list)))

# 方法3：更详细的方法，显示重复项的索引
duplicates_dict = {}
for i, item in enumerate(tj_list):
    if item in duplicates_dict:
        duplicates_dict[item].append(i)
    else:
        duplicates_dict[item] = [i]

print("\n重复项及其位置：")
for item, indices in duplicates_dict.items():
    if len(indices) > 1:
        print(f"{item}: 在位置 {indices} 出现")

重复的项：
XU REN DOU: 出现 2 次

重复项及其位置：
XU REN DOU: 在位置 [501, 502] 出现


In [14]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# select k best features
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, matthews_corrcoef
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score

import warnings
warnings.filterwarnings('ignore')
def bootstrap_ci(metric_func, y_prob, y_true, n_iterations=1000, ci=0.95):
    """
    计算bootstrap置信区间
    """
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    size = len(y_true)
    scores = []
    
    rng = np.random.RandomState(42)  # 设置随机种子
    
    for _ in range(n_iterations):
        # 随机抽样（使用替换）
        indices = rng.randint(0, size, size=size)
        score = metric_func(y_true[indices], y_prob[indices])
        scores.append(score)
    
    # 计算置信区间
    sorted_scores = np.sort(scores)
    alpha = (1 - ci) / 2
    lower_bound = sorted_scores[int(alpha * n_iterations)]
    upper_bound = sorted_scores[int((1 - alpha) * n_iterations)]
    
    return (lower_bound, upper_bound)

def calculate_metrics(y_prob, y_true):
    """
    计算不平衡二分类问题的评价指标，包括95%置信区间
    
    参数：
    y_prob: numpy array, 预测为正类(少数类)的概率值
    y_true: numpy array, 真实标签 (0为多数类，1为少数类)
    
    返回：
    dict: 包含评价指标和置信区间的字典
    """
    # 1. AUC
    auc_score = roc_auc_score(y_true, y_prob)
    auc_ci = bootstrap_ci(roc_auc_score, y_prob, y_true)
    
    # 2. 通过优化F1-score选择最佳阈值
    # precisions, recalls, thresholds = precision_recall_curve(y_true, y_prob)
    # f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    # f1_scores = np.nan_to_num(f1_scores)
    # best_threshold = thresholds[np.argmax(f1_scores[:-1])]
    best_threshold = 0.5
    
    # 使用最佳阈值进行预测
    y_pred = (y_prob >= best_threshold).astype(int)
    
    # 3. F1-score
    f1 = f1_score(y_true, y_pred, zero_division=0)
    f1_ci = bootstrap_ci(lambda y_t, y_p: f1_score(y_t, (y_p >= best_threshold).astype(int)),
                        y_prob, y_true)
    
    # 4. ACC
    acc = accuracy_score(y_true, y_pred)
    acc_ci = bootstrap_ci(lambda y_t, y_p: accuracy_score(y_t, (y_p >= best_threshold).astype(int)),
                         y_prob, y_true)
    
    # 5. MCC
    mcc = matthews_corrcoef(y_true, y_pred)
    mcc_ci = bootstrap_ci(lambda y_t, y_p: matthews_corrcoef(y_t, (y_p >= best_threshold).astype(int)),
                         y_prob, y_true)
    
    # 6. AUPRC
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    auprc = auc(recall, precision)
    
    def auprc_score(y_t, y_p):
        p, r, _ = precision_recall_curve(y_t, y_p)
        return auc(r, p)
    
    auprc_ci = bootstrap_ci(auprc_score, y_prob, y_true)
    
    # 7. Precision
    precisionscore = precision_score(y_true, y_pred)
    precision_ci = bootstrap_ci(lambda y_t, y_p: precision_score(y_t, (y_p >= best_threshold).astype(int)),
                              y_prob, y_true)

    # 8. Recall
    recallscore = recall_score(y_true, y_pred)
    recall_ci = bootstrap_ci(lambda y_t, y_p: recall_score(y_t, (y_p >= best_threshold).astype(int)),
                            y_prob, y_true)

    metrics = {
        'AUC': round(auc_score, 3),
        'AUC_CI': auc_ci,
        'F1': round(f1, 3),
        'F1_CI': f1_ci,
        'ACC': round(acc, 3),
        'ACC_CI': acc_ci,
        'MCC': round(mcc, 3),
        'MCC_CI': mcc_ci,
        'AUPRC': round(auprc, 3),
        'AUPRC_CI': auprc_ci,
        # 'threshold': round(best_threshold, 3),
        'Precision': round(precisionscore, 3),
        'Precision_CI': precision_ci,
        'Recall': round(recallscore, 3),
        'Recall_CI': recall_ci
    }
    return metrics

def format_metrics(metrics):
    """
    格式化指标输出，包含置信区间
    """
    formatted = {}
    for key in metrics:
        if key.endswith('_CI'):
            continue
        if key + '_CI' in metrics:
            formatted[key] = f"{metrics[key]} ({metrics[key+'_CI'][0]}-{metrics[key+'_CI'][1]})"
        else:
            formatted[key] = f"{metrics[key]}"
    
    return formatted

def merge_dicts_to_df(*dicts):
    """
    将多个评估指标字典合并为一个DataFrame
    
    参数:
    *dicts: 多个包含评估指标的字典
    
    返回:
    merged_df: 合并后的DataFrame
    """
    # 创建空的DataFrame保存所有结果
    merged_df = pd.DataFrame()
    
    # 创建索引列表
    new_index = ['train', 'inter_test', 'henan_test', 'kits_test']
    
    # 遍历所有字典
    for i, d in enumerate(dicts, 1):
        # 提取主要指标
        metrics = {
            'AUC': d['AUC'],
            'AUC_CI': f"({d['AUC_CI'][0]:.3f}-{d['AUC_CI'][1]:.3f})",
            'AUPRC': d['AUPRC'],
            'AUPRC_CI': f"({d['AUPRC_CI'][0]:.3f}-{d['AUPRC_CI'][1]:.3f})",
            'F1': d['F1'],
            'F1_CI': f"({d['F1_CI'][0]:.3f}-{d['F1_CI'][1]:.3f})",
            'ACC': d['ACC'],
            'ACC_CI': f"({d['ACC_CI'][0]:.3f}-{d['ACC_CI'][1]:.3f})",
            'MCC': d['MCC'],
            'MCC_CI': f"({d['MCC_CI'][0]:.3f}-{d['MCC_CI'][1]:.3f})",
            'Precision': d['Precision'],
            'Precision_CI': f"({d['Precision_CI'][0]:.3f}-{d['Precision_CI'][1]:.3f})",
            'Recall': d['Recall'],
            'Recall_CI': f"({d['Recall_CI'][0]:.3f}-{d['Recall_CI'][1]:.3f})",
        }
        
        # 转换为DataFrame并添加模型标识
        df = pd.DataFrame([metrics])
        
        # 合并到主DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # 重排列columns
    column_order = ['AUC', 'AUC_CI', 'AUPRC', 'AUPRC_CI', 
                    'F1', 'F1_CI', 'ACC', 'ACC_CI', 'MCC', 'MCC_CI',
                    'Precision', 'Precision_CI', 'Recall', 'Recall_CI']
    merged_df = merged_df[column_order]
    
    # 添加索引列
    merged_df.insert(0, 'Dataset', new_index)
    
    return merged_df

In [68]:
import pandas as pd
# 读取proba和gt

data = pd.read_excel("./result/ring.xlsx")

y_train = data[data['dataset'] == 'train']['ground_truth']
y_val = data[data['dataset'] == 'val']['ground_truth']
y_henan = data[data['dataset'] == 'henan']['ground_truth']
y_kits = data[data['dataset'] == 'kits']['ground_truth']

proba_train = data[data['dataset'] == 'train']['probability']
proba_val = data[data['dataset'] == 'val']['probability']
proba_henan = data[data['dataset'] == 'henan']['probability']
proba_kits = data[data['dataset'] == 'kits']['probability']

# 计算评估指标
result_train = calculate_metrics(proba_train, y_train)
result_val = calculate_metrics(proba_val, y_val)
result_henan = calculate_metrics(proba_henan, y_henan)
result_kits = calculate_metrics(proba_kits, y_kits)
print(result_train)
print(result_val)
print(result_henan)
print(result_kits)

# 合并字典
merged_results = merge_dicts_to_df(result_train, result_val, result_henan, result_kits)

# 保存为CSV文件
merged_results.to_excel('results_ring.xlsx', index=False)

{'AUC': 0.992, 'AUC_CI': (0.9854700854700854, 0.9971321080547675), 'F1': 0.832, 'F1_CI': (0.7450980392156863, 0.8992248062015504), 'ACC': 0.966, 'ACC_CI': (0.9491833030852994, 0.9800362976406534), 'MCC': 0.815, 'MCC_CI': (0.7215331409930178, 0.8901268046829179), 'AUPRC': 0.927, 'AUPRC_CI': (0.8523392426413096, 0.9753783215994241), 'Precision': 0.887, 'Precision_CI': (0.7916666666666666, 0.9636363636363636), 'Recall': 0.783, 'Recall_CI': (0.6716417910447762, 0.8857142857142857)}
{'AUC': 0.709, 'AUC_CI': (0.6087636932707355, 0.8159603554340396), 'F1': 0.242, 'F1_CI': (0.0625, 0.4375), 'ACC': 0.895, 'ACC_CI': (0.8523206751054853, 0.9324894514767933), 'MCC': 0.258, 'MCC_CI': (0.046318664634473566, 0.4602192772174345), 'AUPRC': 0.289, 'AUPRC_CI': (0.1373498465081574, 0.47225314424758846), 'Precision': 0.571, 'Precision_CI': (0.2, 1.0), 'Recall': 0.154, 'Recall_CI': (0.034482758620689655, 0.3157894736842105)}
{'AUC': 0.633, 'AUC_CI': (0.5224483204134367, 0.7285206718346253), 'F1': 0.128, 'F1

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (roc_curve, precision_recall_curve, average_precision_score,
                           roc_auc_score, confusion_matrix, classification_report)
from sklearn.calibration import calibration_curve

def plot_classifier_evaluation_separate(y_true, y_prob, save_path=None):
    """
    分别绘制并保存分类器评估图
    
    参数:
    y_true: 真实标签 (0/1)
    y_prob: 预测概率
    save_path: PDF保存路径
    """
    
    # 1. ROC曲线
    plt.figure(figsize=(8, 6))
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc = roc_auc_score(y_true, y_prob)
    
    plt.plot(fpr, tpr, 'b-', label=f'ROC (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], 'r--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(f'{save_path}_roc.pdf')
    plt.close()
    
    # 2. PR曲线
    plt.figure(figsize=(8, 6))
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    ap = average_precision_score(y_true, y_prob)
    
    plt.plot(recall, precision, 'b-', label=f'PR (AP = {ap:.3f})')
    plt.axhline(y=sum(y_true)/len(y_true), color='r', linestyle='--', label='Random')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(f'{save_path}_pr.pdf')
    plt.close()
    
    # 3. 预测概率分布
    plt.figure(figsize=(8, 6))
    for label, color in zip([0, 1], ['red', 'blue']):
        mask = y_true == label
        plt.hist(y_prob[mask], bins=50, density=True, alpha=0.5, 
                color=color, label=f'Class {label}')
    plt.xlabel('Predicted Probability')
    plt.ylabel('Density')
    plt.title('Probability Distribution')
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(f'{save_path}_prob_dist.pdf')
    plt.close()
    
    # 4. 分类阈值分析
    plt.figure(figsize=(8, 6))
    thresholds = np.linspace(0, 1, 100)
    scores = []
    for thresh in thresholds:
        y_pred = (y_prob >= thresh).astype(int)
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        tp = np.sum((y_true == 1) & (y_pred == 1))
        
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        scores.append([accuracy, precision, recall, f1])
    
    scores = np.array(scores)
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1']
    for i, metric in enumerate(metrics):
        plt.plot(thresholds, scores[:, i], label=metric)
    plt.xlabel('Classification Threshold')
    plt.ylabel('Score')
    plt.title('Metric Scores vs Classification Threshold')
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(f'{save_path}_threshold.pdf')
    plt.close()
    
    # 5. 校准曲线
    plt.figure(figsize=(8, 6))
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)
    plt.plot(prob_pred, prob_true, 's-', label='Calibration curve')
    plt.plot([0, 1], [0, 1], 'r--', label='Perfect calibration')
    plt.xlabel('Mean predicted probability')
    plt.ylabel('Fraction of positives')
    plt.title('Calibration Curve')
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(f'{save_path}_calibration.pdf')
    plt.close()
    
    # 6. 混淆矩阵
    plt.figure(figsize=(8, 6))
    y_pred = (y_prob >= 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    if save_path:
        plt.savefig(f'{save_path}_confusion.pdf')
    plt.close()
    
    # # 7. 分类瀑布图
    # plt.figure(figsize=(12, 6))
    # sorted_indices = np.argsort(y_prob)
    # sorted_probs = y_prob[sorted_indices]
    # sorted_true = y_true[sorted_indices]
    
    # plt.plot(range(len(sorted_probs)), sorted_probs, 'b-', label='Predicted probability')
    # plt.scatter(range(len(sorted_true)), sorted_true, c='r', alpha=0.5, 
    #            label='True label', marker='.')
    # plt.axhline(y=0.5, color='g', linestyle='--', label='Decision threshold')
    # plt.xlabel('Samples (sorted by predicted probability)')
    # plt.ylabel('Probability / Class')
    # plt.title('Classification Waterfall Plot')
    # plt.legend()
    # plt.grid(True)
    # if save_path:
    #     plt.savefig(f'{save_path}_waterfall.pdf')
    # plt.close()
    
    # return {
    #     'AUC': auc,
    #     'AP': ap,
    #     'Confusion Matrix': cm,
    #     'Classification Report': classification_report(y_true, y_pred)
    # }

# 使用示例:
plot_classifier_evaluation_separate(y_val, proba_val, save_path='inter_test')
plot_classifier_evaluation_separate(y_henan, proba_henan, save_path='henan_test')
plot_classifier_evaluation_separate(y_kits, proba_kits, save_path='kits_test')